In [1]:
%load_ext autoreload
%autoreload

In [2]:
#from dipy.data import get_data
from nibabel import trackvis as tv
from dipy.segment.clustering import QuickBundles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

In [5]:
# brn_avg = nib.load("./HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii")
# print(type(brn_avg.header))
# ba_data = brn_avg.get_data()
# print(type(ba_data))
# brn_avg_em = nib.load("./HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500_Eigenmaps.dtseries.nii")
# print(type(brn_avg_em))
sub_100206_d50 = nib.load("./3T_HCP1200_MSMAll_d50_ts2_Z/100206.dtseries.nii")
sd50 = sub_100206_d50.get_data()
print(sd50.shape)
print(type(sub_100206_d50))

(50, 91282)
<class 'nibabel.cifti2.cifti2.Cifti2Image'>


In [8]:
from dipy.segment.clustering import QuickBundles as qbs

In [10]:
qb = qbs(threshold=10)
clusters = qb.cluster(sd50)

ValueError: Buffer has wrong number of dimensions (expected 2, got 1)

In [3]:
import dipy.reconst.dti as dti

In [5]:
white_matter = (ba_data == 1) | (ba_data == 2)

In [ ]:
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model

csa_model = CsaOdfModel(gtab, sh_order=6)
csa_peaks = peaks_from_model(csa_model, data, default_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=45,
                             mask=white_matter)